In [64]:
import torch
import json
import transformer as t
import tokenizer as tokenizer
import importlib
importlib.reload(t);
importlib.reload(tokenizer);

In [73]:
#
# Hyper parameter
#
topic           = 'commits'
batch_size      = 64
max_iterations  = 4000
eval_interval   = 1000
learning_rate   = 1e-4
eval_iters      = 200

#
# Network 
#
t.attention_heads_per_block = 8
t.attention_blocks          = 16
t.sample_size               = 32     # number of consecutive characters to predict from
t.embedding_size            = 256    # size of the embedding vectors
t.dropout                   = 0.2

# remember to set t.vocabulary_size when data is loaded

In [68]:
# read in file
with open(f'{topic}/{topic}.txt', 'r', encoding='utf-8') as f:
    text = f.read()

decoder, tokens = tokenizer.tokenize(text, 100)
t.vocabulary_size = len(decoder)
decode = lambda l: ''.join([decoder[i] for i in l])

# save vocabulary
dictionary = {str(k): v for k, v in decoder.items()}
with open(f'{topic}/vocabulary.json', 'w') as file:
    json.dump(dictionary, file)

# create data tensors
data = torch.tensor(tokens, dtype=torch.long)
n = int(0.9*len(data))
training_data = data[:n]
validation_data = data[n:]

In [74]:
#
# Create model
#
torch.manual_seed(1337)
model = t.Transformer()
m = model.to(t.device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
start_iteration = 0
sum(p.nelement() for p in model.parameters())

12738254

In [ ]:
#
# Load model from checkpoint
#
start_iteration = 2001

# Load the states
checkpoint = torch.load(f'{topic}/{topic}-{start_iteration-1}.nn');
model.load_state_dict(checkpoint['model_state_dict']);
optimizer.load_state_dict(checkpoint['optimizer_state_dict']);

model.train()

In [75]:
#
# mini-batch creation
#
def get_batch(split):
    data = training_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - t.sample_size, (batch_size, ))
    x = torch.stack([data[i:i+t.sample_size] for i in ix])
    y = torch.stack([data[i+1:i+t.sample_size+1] for i in ix])
    x,y = x.to(t.device), y.to(t.device)
    return x,y
#
# evaluation
#
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            _, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

def save_model(step):
    model.eval()
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f'{topic}/{topic}-{step}.nn');
    model.train()

def export_model(step):
    model.eval()
    dummy_input = torch.randint(low=0, high=t.vocabulary_size, size=(1, t.sample_size), dtype=torch.long)
    torch.onnx.export(model, dummy_input, f"{topic}/{topic}-{step}.onnx");
    model.train()    

In [76]:
#
# Training
#
model.train()
for iter in range(max_iterations):
    current_iteration = iter + start_iteration
    if current_iteration % eval_interval == 0 and (start_iteration == 0 or current_iteration > start_iteration):
        print(f"{current_iteration}: evaluating...")
        losses = estimate_loss()
        print(f"{current_iteration}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")    

    if current_iteration % 1000 == 0 and (start_iteration == 0 or current_iteration > start_iteration):
        print(f"{current_iteration}: saving checkpoint...")
        save_model(current_iteration);
        export_model(current_iteration);
        print(f"{current_iteration}: checkpoint saved.")

    if current_iteration % 250 == 0:
        print(f"{current_iteration}: training")

    xb, yb = get_batch('train')
    _, loss = model(xb,yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

current_iteration += 1
print(f"{current_iteration}: evaluating...")
losses = estimate_loss()
print(f"{current_iteration}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
print(f"{current_iteration}: saving checkpoint...")
save_model(current_iteration)
export_model(current_iteration)
print(f"{current_iteration}: checkpoint saved.")

0: evaluating...


### Commit message generation training

#### Hyper and network parameters
| Parameter                 | Value 
| :--------                 | ----:
| tokenizer steps           | 100
| sample size               | 32
| embedding size            | 256
| batch size                | 64
| learning rate             | 1e-4
| attention heads per block | 8
| attention blocks          | 16
| dropout ratio             | 0.2


#### Training results
| Iteration | Loss (training)   | Loss (validation)
| :-------: | :-------------:   | :---------------:
| 0         | 8.2615            | 8.2457 


In [19]:
model.eval()
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=t.device), max_tokens=500)[0].tolist()))

GFILES
1not in TFS -> Fixed debug when Oid-Event for OQR ChangeErpDeviceCallback Referenz behoben
Parameternalen Logging-Ausgaben
Kommentar
ToDn-Sufrter Konmentarbe Optimierung (nur im Looltips und die SapDevice-Antragung der Rows2, Tests
Handlbar noch bei "AktiveServicesMsi - und version 3.03 5.9.1.sql to 4.3
Configuration to text to the repository
using of DriverInityContextSpoolerConfigVersionsGetPrintServerServerChecks tests + Parameter layout bei ActVield
Implementedmodus
PrintServer implementiert
Interfaces für Benutzerverwaltung
Serverd-Systeme von der Loggings aus Start, Default einfachen
Unterschlüsselung
Nochmal bei prüft sind
Control
Leerzeichen aktualisieren (noch können
BuyeeXml-Namen ChangesAction öffnen auf 6.5 gefixt
Fehler in der Eigenschaftendialog entfernt
Versionsdialog verwendet specipatibility abgestellt
DataContainer.gets auf Usage-Tests für hidlichbar, behavior problem besser angepasst
Code und und zum Letzten angepasst beid wird erstellt geändert, während der D